In [3]:
import cv2
import wave
import pyaudio
import threading
import time
import speech_recognition as sr

# Function to handle live video feed with audio recording
def live_video_with_audio_recording(output_audio_path, record_time=10):
    def record_audio():
        """Record audio from the microphone."""
        chunk = 1024
        sample_format = pyaudio.paInt16
        channels = 1
        rate = 16000

        p = pyaudio.PyAudio()
        stream = p.open(format=sample_format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)

        frames = []
        for _ in range(0, int(rate / chunk * record_time)):
            data = stream.read(chunk)
            frames.append(data)

        stream.stop_stream()
        stream.close()
        p.terminate()

        # Save recorded audio to a file
        with wave.open(output_audio_path, 'wb') as wf:
            wf.setnchannels(channels)
            wf.setsampwidth(p.get_sample_size(sample_format))
            wf.setframerate(rate)
            wf.writeframes(b''.join(frames))

    # Start video feed
    capture = cv2.VideoCapture(0)

    # Record audio in a separate thread
    audio_thread = threading.Thread(target=record_audio)
    audio_thread.start()

    while audio_thread.is_alive():  # Keep showing video feed while recording audio
        ret, frame = capture.read()
        if not ret:
            break
        cv2.imshow("Live Video Feed", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    audio_thread.join()
    capture.release()
    cv2.destroyAllWindows()

# Function to transcribe audio to text
def audio_to_text(audio_path):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_path) as source:
            audio = recognizer.record(source)
        text = recognizer.recognize_google(audio)
        print("Extracted Words:", text)
        return text
    except sr.UnknownValueError:
        return "Could not understand the audio."
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"

# Main Execution for Live Video
if __name__ == "__main__":
    time.sleep(5)  # Countdown before starting
    audio_file_live = "live_audio.wav"
    live_video_with_audio_recording(audio_file_live, record_time=10)
    audio_to_text(audio_file_live)

Extracted Words: so I have a fever and I also I am feeling dizzy so can you please help me with that
